# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [15]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching potential venues in San Francisco that could meet the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference venues in San Francisco"} 


Search results: Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechusa.com/
Snippet: The World Agri-Tech Innovation Summit is the stand out event with curated leaders, innovators and investors from across the AgTech landscape.
---
Title: AI Industry Conference Venue in SF
Link: https://ssfconf.com/ai-conference-venue-sf/
Snippet: The South San Francisco Conference Center is the premier destination for AI summits, expos, and networking events, offering exceptional space, ...
---
Title: IT & Technology Events in Sa

This is the agent final answer: Venue: South San Francisco Conference Center
Location: 255 S Airport Blvd, South San Francisco, CA 94080
Phone: (650) 877-8787
Website: https://ssfconf.com/ai-conference-venue-sf/

The South San Francisco Conference Center offers:
- Expansive, Tech-Ready Event Space with over 20,500 square feet
- Unparalleled Accessibility, located just 5 minutes from San Francisco International Airport (SFO)
- Secure and Professional Environment with top-tier security services
- Advanced AV capabilities, high-speed internet, and streaming options
- Flexible room configurations for keynote sessions, workshops, and networking lounges
- Convenient access to hotels, public transit, and major highways
- Dedicated event coordination team for seamless event planning and support

This venue seems to meet the criteria for the Tech Innovation Conference in terms of space, technology infrastructure, security, and accessibility.
Please provide a feedback: Yes, this options are good

Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Read website content
Action Input: {"website_url": "https://www.techinnovationconference.com"} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.techinnovationconference.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f6caf54ab50>: Failed to resolve 'www.techinnovationconference.com' ([Errno -2] Name or service not known)")).
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference 2025 details"} 

I tried reusing the same input, I m

Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Read website content
Action Input: {"website_url": "https://techconnectworld.com/"}I need to find the appropriate venue for the event with catering and equipment setup for 500 participants on 2024-09-15 at the South San Francisco Conference Center.

Action: Read website content
Action Input: {"website_url": "https://www.ssfconf.com/"} 

South San Francisco Conference Center | Event and Meeting Spaces In The Bay Area -
Skip to content
Meetings Overview Promotions / Packages Floor Plans Catering Services Audio/Visual Event Services Request for Proposal Media GalleryCelebrations Overview Promotions / Packages Floor Plans Catering Services Audio/Visual Event Services Ask For A Quote Media GalleryLodgingBay Area Overview Our Community Attractions Transportation OptionsContactGeneral Info Overview Conference Center Authority Rules & Regulations Safe & Secure Events 

Exception in thread Thread-9 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Read website content
Action Input: {"website_url": "https://www.techconnectworld.com/"} 

TechConnect World

Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Read website content
Action Input: {"website_url": "https://www.techconnectworld.com/"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference 2025 official website"} 


Search results: Title: TechConnect World 2025
Link: https://www.techconnectworld.com/
Snippet: Join us in Austin in 2025 for the world's largest gathering of commercializationfocused innovators. Make meanin

Thought: I need to gather information about the Tech Innovation Conference and engage potential attendees effectively.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference 2025 official website"} 


Search results: Title: TechConnect World 2025
Link: https://www.techconnectworld.com/
Snippet: Join us in Austin in 2025 for the world's largest gathering of commercializationfocused innovators. Make meaningful connections and drive your business forward.
---
Title: 2025 TechConnect World Innovation Conference and Expo
Link: https://techconnect.org/2025-techconnect-world-innovation-conference-and-expo/
Snippet: Join the world's brightest researchers, innovators, and technology scouts as they convene in Austin, Texas from June 9-11, 2025. JOIN US. previous next
---
Title: CES - The Most Powerful Tech Event in the World
Link: https://www.ces.tech/
Snippet: CES 2025: The Global Stage for Innovation, Connecting the World, Creating the Future · Consumer Technol

Final Answer: 

# Tech Innovation Conference 2025 Marketing Report

## Event Details:
- **Name:** TechConnect World 2025
- **Date:** June 9-11, 2025
- **Location:** Austin, Texas
- **Website:** [TechConnect World](https://www.techconnectworld.com/)

## Introduction:
Welcome to the TechConnect World 2025, the world's largest gathering of commercialization-focused innovators. Join us in Austin to make meaningful connections and drive your business forward.

## Marketing Activities:
1. **Search Engine Optimization (SEO):** Utilize targeted keywords related to technology innovation and conferences to increase online visibility.
2. **Social Media Campaigns:** Engage with potential attendees on platforms like LinkedIn, Twitter, and Facebook to promote the event.
3. **Email Marketing:** Send out personalized invitations and updates to a curated list of potential attendees.
4. **Collaborations:** Partner with industry influencers and organizations to expand reach and credibility.
5. **Content 

- Display the generated `venue_details.json` file.

In [16]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '255 S Airport Blvd, South San Francisco, CA 94080',
 'booking_status': '',
 'capacity': 0,
 'name': 'South San Francisco Conference Center'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [17]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Tech Innovation Conference 2025 Marketing Report

## Event Details:
- **Name:** TechConnect World 2025
- **Date:** June 9-11, 2025
- **Location:** Austin, Texas
- **Website:** [TechConnect World](https://www.techconnectworld.com/)

## Introduction:
Welcome to the TechConnect World 2025, the world's largest gathering of commercialization-focused innovators. Join us in Austin to make meaningful connections and drive your business forward.

## Marketing Activities:
1. **Search Engine Optimization (SEO):** Utilize targeted keywords related to technology innovation and conferences to increase online visibility.
2. **Social Media Campaigns:** Engage with potential attendees on platforms like LinkedIn, Twitter, and Facebook to promote the event.
3. **Email Marketing:** Send out personalized invitations and updates to a curated list of potential attendees.
4. **Collaborations:** Partner with industry influencers and organizations to expand reach and credibility.
5. **Content Creation:** Develop engaging and informative content about the conference to attract participants.

## Attendee Engagement:
1. **Interactive Sessions:** Offer interactive workshops, panel discussions, and Q&A sessions to keep attendees engaged.
2. **Networking Opportunities:** Facilitate networking sessions and one-on-one meetings to encourage connections.
3. **Virtual Experience:** Provide a seamless virtual experience for remote attendees with live streaming and virtual booths.
4. **Feedback Mechanism:** Gather feedback during and after the event to improve future conferences and enhance attendee satisfaction.

## Conclusion:
The TechConnect World 2025 promises to be a transformative event for technology innovators. By implementing a robust marketing strategy and engaging potential attendees effectively, we aim to maximize event exposure and participation, ensuring a successful and impactful conference experience.